# Multisesh
Multisesh is a python package which aims to simplify the processing and analysis of complex microsopy datasets by:
* standardising image and metadata loding across microscope data file types
* allowing the combination of arbitrary numbers and types of datasets into one organised object - 'multiple-sessions'
* allowing any arbitrary section (z-slice, channel, field etc) of data to be loaded independently (aiding memory management and algorithmic processing)
* providing simple in-notebook data viewing
* providing common processing functions (and easily extensible)
* allowing saving and reopening of processed image data (i.e. for saving processing image results or mid-points in analysis)

# Multisesh structure details

Multisesh works via a simple hierarchical structure of 3 python objects:
* **XFold** - 'experiment folder'
  * lightweight object containing organised info about all data in the folder you specify
  * there are some functions that act on this to just process the whole folder (though then you have less control analysis)
  * all data in the folder is organised into 'Sessions' which are stored in the XFold...
<br>
<br>
* **Session**
  * there is one of these 'Session' objects to represent each imaging session found in the provided folder
  * by imaging session we mean one imaging protocol run by a microscope
    * can think of it essentially as each time the user has pressed 'go'
    * i.e. it can be multiple files if this how the microscope saves an imaging session
    * ...or one file can contain multiple Sessions, e.g. .lif format sometimes does that
    * the key definition is one Session must be fully described by one metadata, e.g. with one shape (no. of time points/z-slices/channels...)
    * more technically: the data of one Session must be non-jagged so could fit in a standard numpy array
  * all Sessions are found in the list: xfold.SessionsList (ordered by the time they were aquired)
  * this Session object holds the metadata, e.g. Session.Shape, ...
  * otherwise it is also a lightweight object with no actual pixel data in it
<br>
<br>
* **TData** - 'the data' :)
  * this is the object that contains your data: TData.data is a numpy array of your data
  * also contains all the relevant metadata for that part of the image data
  * you build a TData from an XFold or Session, specifying what bit of the Session you want: tdata = XFold.makeTData(S=,Z=..,C=...)
  * where S is the Session number
  * the data is always in the same 7D format: (T,F,M,Z,C,Y,X)
    * T: time-points
    * F: fields (e.g. multi-position images)
    * M: montage tiles
    * Z: z-slices
    * C: channels
    * Y,X: 2D image coordinates
  * functions apply directly to this object to change it, e.g. TData.zProject(),TData.SaveData()...

# Environment setup

### Basic (no YOLOSAM or Cellpose)

create basic environment:

```console
conda create -n multisesh python=3.11.4
conda activate multisesh
```
then navigate to the package folder (where pyproject.toml is located):

```console
pip install -e .
```

That should be everything!

### Environment allowing YOLOSAM

```console
conda create -n yolosam jupyterlab 
conda activate yolosam
```
then copy commnd to install pytorch for your system from here: https://pytorch.org/get-started/locally/
usually something like:

```console
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
```

then some others we need:

```console
conda install conda-forge::segment-anything 
conda install conda-forge::ultralytics 
conda install anaconda::dill
```

then navigate to the package folder (where setup.py is located):

```console
pip install -e .
```

That should be everything! There is one little error that currently shows during the pip install but it seems to work fine

# Load experiment

supported datatypes/microscopes so far:
* czi
* lif
* nd2
* Opera
* Incucyte
* Andor
* OME
* Micromanager
* General tiffs

BUT: within these filetypes there are some things that aren't supported yet. E.g. I haven't ever worked with a lif file containing multiple time-points so haven't been able to see how to read that properly. These things are fast to add so let me know if you have examples!

### Import multisesh

In [1]:
import multisesh as ms

### Basic experiment loading

In [1]:
root = r'./RawData' # put all your data in one folder and set this as the path (doesn't support multiple locations yet but wouldn't be hard fix)
xfold = ms.XFold(root) # build your XFold object
xfold.Summarise() # this prints a basic report of data it found

### Faster experiment loading for large datasets

If you have a large dataset that is taking more than a minute or so to load try this option. Multisesh looks in every file to check dimensions. Some microscopes save each frame separately so you end up with thousands of files so this takes ages. If they all have the same dimensions you can use this option to speed things up by skipping the checking.

In [ ]:
import multisesh as ms

root = r'./RawData' # put all your data in one folder (doesn't support multiple locations yet but wouldn't be hard fix)
xfold = ms.XFold(root,assumeConstantDims=True) # build your XFold object
xfold.Summarise() # basic report of data it found

### Further loading options

Other options when loading include (see \__doc__ for more details):
* FieldIDMapList - let's you give names to fields
* StartTimes - allows you to set a global time marking the start of the experiment (or e.g. when a compound was added) - can have different times for different Fields
* Filters - ignore (i.e. don't load) files which contain any of the strings provided in this list

In [ ]:
root = r'./RawData' 

fieldIDMapList = [...] # give fields names, they will be used for saving and can be used instead of indices
starttimes = [...] # set a global time time zero for the experiment (or differenet time zeros for different fields) 
filters = [] # ignore any files in root that contain any strings in this list

xfold = ms.XFold(root,FieldIDMapList=fieldIDMapList,StartTimes=starttimes,Filters=filters) # build your XFold object
xfold.Summarise() # basic report of data it found

# Load image data from experiment 

load image data by specifying the session S and section (T,F,M,Z,C)

In [ ]:
tdata = xfold.makeTData(S=0,F=[0,4,7])

In [ ]:
# an alternative way is to make a TData a Session, e.g. the Sessions stored in the XFold
tdata2 = xfold.SessionsList[0].makeTData(F=[0,4,7])

# Visualise data

to enable data visualisation you may need to install widgets into jupyter notebooks:
* "View" -> "Extension Manager"
* on "Warning" and click "Yes"
* click the search box and enter "jupyterlab-widgets" and click search
* Install

In [ ]:
tdata.Plot()

# Segmentation

first example here applies YOLOSAM segmentation to all the data in the xfold

In [ ]:
diameter = 14 # an estimate for the diameter of nuclei in your data in μm
nuc_chan =0 # the index of the channel you want to segment (currently needs to be same channel in all Sessions)
clearborder = True # where to remove moasks that touch image edge
returnSegs = False # whether to return segmentations as a list of numpy arrays (one for each Session)
outName = 'NuclearSegmentations' # the name of the folder to save the segmentations to (will be saved next to the raw data folder)

xfold.YOLOSAM(diameter,nuc_chan,clear_borderQ=clearborder,returnSegmentations=returnSegs,saveMasks=outName)

this example applies YOLOSAM segmentation to one tdata... this is useful if you want to process the tdata in some way before the segmentation

In [ ]:
diameter = 14 # an estimate for the diameter of nuclei in your data in μm
nuc_chan =0 # the index of the channel you want to segment within your tdata (check channel names with tdata.Chan)
clearborder = True # where to remove moasks that touch image edge
returnSegs = False # whether to return segmentations as a numpy array
addSeg = True # where to add segmentation as a new channel in the tdata

tdata.YOLOSAM(diameter,nuc_chan,clear_borderQ=clearborder,returnSegmentations=returnSegs,addSeg2TData=addSeg)

# since we added the segmentation to the tdat we can new view it
tdata.Plot(plotSegMask=True)

# Example processing routines

this example saves z-projected vesions of the data with a time label added:

In [ ]:
root = r'/Volumes/Pure6000/Users/jdelarosa/opera/20240327_HUVECs_20x__2024-03-27T17_48_56-Measurement 1/Images'

# this will be the time zeros that the labels we add count from 
stimulationMoment = datetime.datetime(2024, 3, 8, 10, 32, 18) + datetime.timedelta(minutes=18)

xfold = ms.XFold(root,StartTimes=stimulationMoment) 

outDir = 'LabelledVideos'

%%time
for i,s in enumerate(xfold.SessionsList): # loop through all Sessions in the folder
    print("i: ",i)
    for f in range(s.NF): # we loop through all fields in the Session
        print('f: ',f)
        tdata = s.makeTData(F=f) # loading just one field at a time avoids memory problems
        tdata.zProject() # do a maximum projection along z
        tdata.LabelVideo(roundM=1,style="mm:ss",label_size_divisor=20) # add a label of the time in the bottom right corner of the image
        tdata.SaveData(outDir) # note that default behaviour is to divide save files into directories according to field

here we reload the above data and segment the nuclei

In [ ]:
roots = './XFoldAnalysis_LabelledVideos'
xfold2 = ms.XFold(root2)
diameter = 14 

nuc_chan =0 
clearborder = True 
returnSegs = False 
outName = 'NuclearSegmentations'

xfold2.YOLOSAM(diameter,nuc_chan,clear_borderQ=clearborder,returnSegmentations=returnSegs,saveMasks=outName)

now we load the segmented nuclei to track them through time
(i.e. currently the pixel value of the each value is its label but this value changes from time-point to time-point.. this, where possible, gives the same nucleus mask the same label across time points.

In [ ]:
root3 = r'./XFoldAnalysis_NuclearSegmentations'
xfold3 = ms.XFold(root3)

track_out_name = 'TrackedNuclearSegmentations'

xfold3.trackSegmentations(track_out_name,removeSmall=1000,saveMasks=True,sessions=[0])

in this one we correct a dataset for inhomogenous field-of-view brightness (flat-field correction)

an important consideration is that the correction is calculated from all the data in the tdata (separated by channel). So you want enough images to calculate a reliable correction, but not too much to run into memory/time problems. Here we load just one time-point of the data at a time, but load all fields/montage-tiles/z-slices

In [ ]:
root = './RawData'
xfold = ms.XFold(root)

outName = 'FlatFieldCorrected'

for s in xfold.SessionsList:
    for t in range(s.NT):
        tdata = s.makeTData(T=t)
        tdata.BaSiCHomogenise()
        tdata.SaveData(outName)

here we stabilise a video (i.e. remove x-y drift/movements). The function is a bit limited still but worked well on the first datasets I tried

In [ ]:
root = './RawData'
xfold = ms.XFold(root)

outName = 'Stabilised'

for s in xfold.SessionsList:
    for f,m in product(range(s.NF),range(s.NM)): # loop over all fields and montage tiles
        tdata = s.makeTData(F=f,M=m,C='BF') # note we load just the bright field channel, by name rather than index
        tdata.zProject()
        tdata.BaSiCHomogenise()
        tdata.SaveData(outName)